# Hierarchical Latent Dirichlet Allocation (HLDA) Topic Modeling Algorithm Introduction

### Joseph Jinn and Keith VanderLinden

This Jupyter Notebook file provides a very simple high-level overview of the HLDA topic modeling algorithm.  We briefly discuss the plate notation diagram, pseudocode, and statistical formula for the model.

### Plate Notation for the HLDA Algorithm:

![hlda](../images/hlda_model.png)

The right half of the plate diagram depicts the base Latent Dirichlet Allocation (LDA) algorithm.  The left half of the plate diagram depicts the "hierarchical" part that makes it HLDA utilizing the nested Chinese Restaurant Process (nCRP).  Latent (hidden) variables are un-shaded whereas observed (visible) variables are shaded.

$\Upsilon$ - parameter controlling how often a document chooses a new path.

$T$ - collection of infinite # of L-level paths drawn from a nCRP. (tree-like structure)

$c+{d}$ - path of document 'd'.

$\alpha$ - Dirichlet distribution prior on the document-topic distribution.

$\eta$ - Dirichlet distribution prior on the word-topic distribution.

$\theta$ - author-topic distribution.

$\beta$ - word-topic distribution.

$z$ - the topic assigned to the word.

$w$ - the selected word.

$N$ - the entire vocabulary of words in a document "d".

$M$ - the entire collection of documents in the corpus.

$\infty$ - infinity # of restaurants (topics).

### Chinese Restaurant Process:

**Note: The below is essentially copy/paste from one of my references besides some Latex, Markdown, and minor adjustments.**

The Chinese Restaurant Process (CRP) is a distribuition on partitions of integers. Imagine there are M customers in a Chinese restaurant with infinite tables. The first customer sits at the first table. Customers have the following choices:

* Sit in the table that some one alse is already there
* Sit in a new table

These two choices have probabilities that depend on the previosu customers at the tables. 

Specifically, for the $m$th customer, the probability to sit in a table is:

- p(occupied table i| previous customers) = $\frac{m_i}{\gamma+m-1}$
- p(next unoccupied table| previous customers) = $\frac{\gamma}{\gamma+m-1}$,

where $m_i$ represnets the number of previous customers at the table $i$; $\gamma$ is a parameter.

If we have M customers, the CRP will give us a partion of M customers, which has the same structure as a Dirichlet process.

![CRP](../images/chinese_restaurant_process.png)

### Nested Chinese Restaurant Process:

**Note: The below is essentially copy/paste from one of my references besides some Latex, Markdown, and minor adjustments.**

The CRP establishes a one-to-one relationship between tables and mixture components. A hierarchical version of CRP was also developed to model one-to-many. The process like this is called nested Chinese Restaurant Restaurant Process (nCPR).The nCRP is very similar with CRP except for its hieracrchical structure.

We can see an example in the following plot To help understand the nCRP.

![nCRP](../images/nested_chinese_restaurant_process.png)

Suppose a traveller came to a new city and wanter to try the restaurants there. There is a root restaurant, which is the first stop for new travellers. He came to the root restaurant and chose a table based on the Chinese restaurant process we described before. And each table has a card that references to the next restaurant. The traveller followed the card's instruction and went to the restaurant on the card. Then he chose a table in the second restaurant followed the CRP. In a conclusion, each traveller has a path that contains a batch of restaurants and each restaurant represents a level of the topic model.

### Statistical Formulas and Explanation of the Gibbs Sampling Process for the HLDA Algorithm:

**Note: The below is essentially copy/paste from one of my references besides some Latex, Markdown, and minor adjustments.  I do not pretend to have sufficient statistical background to understand all of this.**

#### Approximate inference by Gibbs sampling

<br>

##### Introduction to Gibbs sampling:

Gibbs sampling is commonly used for statistical inference to determine the best value of a parameter. The idea in Gibbs sampling is to generate posterior samples by sweeping through each variable (or block of variables) to sample from its conditional distribution with the remaining variables fixed to their current values. For instance, the standard step for Gibbs sampling over a space of variables a, b, c is:

* Draw a conditioned on b, c.
* Draw b conditioned on a, c.
* Draw c conditioned on a, b This process continues until “convergence”, which means that the sample values have the same distribution as if they were sampled from the true posterior joint distribution.

##### Gibbs sampling for the hLDA model:

The variables that are needed to be sampled are:

1. $w_{m,n}$: the $n$th word in the $m$th document (Important note: these are the only observed variables in the model)

2. $c_{m,l}$: the restaurant (node), the $l$th topic in the $m$th document

3. $z_{m,n}$: the assignment of the $n$th word in the $m$th document to one of the $L$ topics

There are also some variables needed in the model, but they are not needed to be sampled.

After illustrating the variables in the model, we also need to know the order and the methods of the sampling. We can apply the sampling methods into two steps:

1. sample the $z_{m,n}$ variale by using LDA+CRP
2. sample the $c_{m,l}$ based on the first step (given the LDA hidden variables).


#### To be more specific:


##### Sample $z_{m,n}$:

The $z_{m,n}$ is sampled under LDA model based on the method in paper (Dirichlet integrals):

\begin{align*} p(z_{i}=j\hspace{0.5ex}|\hspace{0.5ex}{\bf z}{-i},{\bf w})\propto\frac{n{-i, j}^{(w_{i})}+\beta}{n_{-i, j}^{(\cdot)}+W\beta}\frac{n^{(d_{i})}+\alpha}{n_{-i,\cdot}^{(d_{i})}+T\alpha} \end{align*}

where:

$z_{i}$ is the assignments of words to topics;

$n_{-i,j}^{(w_{i})}$ is number of words assigned to topic $j$ that are the same as $w_i$;

$n_{-i,j}^{(\cdot)}$ is total number of words assigned to topic $j$;

$n_{-i,j}^{(d_{i})}$ shows number of words from document $d_i$ assigned to topic $j$, $n_{-i,\cdot}^{(d_{i})}$ represents total number of words in document $d_i$;

$W$ shows number of words have been assigned.

$\alpha,\beta$: free parameters that determine how heavily these empirical distributins are smoothed.


##### Sample $c_m$ from the nCRP:

The conditional distibution for $c_m$:

$p(w_m|c,w_{-m},z)$: the likelihood of the data given a particular choice of $c_m$

$p(c_m|c_{-m})$: the prior on $c_m$ implied by the nested CRP

$$p(c_m | w, c_{-m}, z) \propto p(w_m | c, w_{-m}, z) p(c_m | c_{-m})$$


The calculation of the $p(w_m | c, w_{-m},z)$ value based on the likelihood function:


$$p(w_m | c, w_{-m},z) = \prod_{l=1}^{L} (
\frac{
   \Gamma (n_{c_{m,l,-m}}^{(\cdot)}+W\eta)}{\prod_{\omega} 
   \Gamma (n_{c_{m,l,-m}}^{(\omega)}+\eta)}
\frac{\prod_{\omega} \Gamma(n_{c_{m,l,-m}}^{(\omega)}+n_{c_{m,l,m}}^{(\cdot)}+\eta)}{\Gamma(n_{c_{m,l,-m}}^{(\cdot)}+ n_{c_{m,l,m}}^{(\cdot)} W\eta)})$$

where,

$n^{(w)}_{c_m,I,-m}$: the number of word $w$ that have been assigned to the topic indexed y $c_{m,l}$ not including those in the current document

$W$: the total vocabulary size

$\eta$: free parameter


==============================================================================================================

![hlda equation](../images/hlda_equation.png)

Conditional distribution for $c_{m}$, the 'L' topics associated with document 'm'.

![hlda equation](../images/hlda_equation_2.png)

Probability function given a specific choice of $c_{m}$ and $p(c_{m} | c_{-m})$ as the prior on $c_{m}$ implied by nCRP.

### Pseudocode for the HLDA algorithm:

This is a graphical version of the pseudocode for the algorithm.

![hlda pseudocode](../images/hlda_pseudocode.png)

This is the text version of the pseudocode which is equivalent to the graphical version above.

![hlda pseudocode](../images/hlda_pseudocode_2.png)

### Explanation of the process:

Note: This was obtained from the README.md for a GitHub Repository containing another implementation of the HLDA algorithm in Python.

For a hierarchial topic model with L-levels, we can imagine it as a L-level tree and each node presents a topic.

Generation of a document:

* Choose a path from the root to a leaf
* Choose the topic proportions $\theta$ from a L-dimension Dirichlet
* Generated the words in the document for m a mixture of the topics along the path from the root to leaf, with mixing proportions $\theta$

<br>

![hlda explanation](../images/hlda_explanation.png)

<br>

For the $m$th document in the corpus:

* Let $c_1$ be the the root restaurant
* For each level $l \in {2,. . ., L}$: 1. Draw a table from the restaurant $c_{l-1}$. 2. Set $c_l$ to be the restaurant referred to by that table
* Draw an $L$-dimensional topic proportion vector $\theta$ for $Dir (\alpha)$
* For each word $n\in {1, . . ., N}$: 1. Draw $z \in {1,. . . ,L}$ from $mult (\theta)$; 2. Draw $w_n$ from the topic assoicated with the restaurant $c_z$

## A Simplified Hierarchical Latent Dirichlet Allocation Topic Modeling Algorithm Example:

Placeholder.

**TODO - implement simple hand-worked example of one iteration through the algorithm (provided we can find an example)** 

### Completion of the FIRST iteration of the HLDA algorithm:

Rinse and repeat.

## Resources Referenced:

- https://en.wikipedia.org/wiki/Greek_alphabet
    - for Greek alphabet name reference.
    

- https://en.wikipedia.org/wiki/Dirichlet_integral
    - apparently the statistics involved utilize these.


- https://www.aclweb.org/anthology/W14-3111
    - has a nice overview of the HLDA algorithm with image.
    
    
- https://github.com/sakuramomo1005/STA663_Final_Project
    - GitHub page that helps interpret the plate diagram; explains HLDA Gibbs sampling; explain nCRP; etc.
    - This reference is has proved very useful.
    

- https://www.semanticscholar.org/paper/Guided-HTM%3A-Hierarchical-Topic-Model-with-Dirichlet-Shin-Moon/9954048379a85148f0ae4698b84c51f0ef897e2e/figure/4
    - image helped explain what $\Upsilon$ represented.
    
    
- https://cs.brown.edu/courses/csci2950-p/fall2011/lectures/2011-10-25_bryantWang.pdf
    - another source for pseudocode; provides explanation of the statistical components; presentation slides.
    
    
- https://www.researchgate.net/publication/261201482_HLDA_based_text_clustering
    - utilizing Figure 1 as representative of the pseudocode.
    